# DATA analysis and pipelines

This notebook accompanies a project on Independent Componennt Analysis (ICA) applied to electroencelographic data. Depends on "Data Cleaning.ipynb" and "utils.py". 

## Load packages and data.

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import FastICA
from coroica import CoroICA, UwedgeICA

from scipy.special import comb
from scipy.signal import butter, lfilter

from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from utils import RFsLDA

from timeit import default_timer
from tqdm.notebook import tqdm
from itertools import combinations

In [ ]:
#Number of trials in each session
n_ses = [273, 281, 270, 283, 270, 273, 262, 228, 262, 276, 219, 215, 271, 277, 264, 271, 237, 264]

#Number of components. Should be 22 if data is not preprocessed with CAR.
n_c = 21 

In [ ]:
#Load the data
Classes = np.load('Classes.npy')
Signals = np.load("Signals.npy")

print(Classes.shape, Signals.shape, Signals.dtype)

In [ ]:
#Setup parameters for 8-30Hz bandpass filtering
lowcut = 8
highcut = 30
fs = 250 # sampling frequency
order = 3
nyq = 0.5 * fs
low = lowcut / nyq
high = highcut / nyq
b, a = butter(order, [low, high], btype='band')

# Try sanitycheck classification on raw data

In [ ]:
X_test = np.log(np.var(Signals, axis = 2)) #Extract features

In [ ]:
#Fit classifier
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_test, Classes)

In [ ]:
#Get prediction accuracy
predicts = clf.predict(X_test)
np.mean(predicts==Classes)

# Pipeline

In [ ]:
#Define functions that will extract correct indicies for a particular train-test split.
N_list = range(sum(n_ses))

def Indices(persons):
    ans = []
    for p in persons:
        a = sum(n_ses[:2*p])
        b = sum(n_ses[:2*p+2])
        ans += list(range(a,b))
    return ans

def reshape_signal(arr):
    x,y,z = arr.shape
    a = np.stack(arr, axis = 1)
    a = a.reshape((y, x*z))
    return a.transpose()

def Signals_pipeline(persons):
    I = Indices(persons)
    LeaveOut = list(set(N_list) - set(I))

    ## Get signals and reshape
    Signals1,Signals2 = Signals[I], Signals[LeaveOut]
    return reshape_signal(Signals1),reshape_signal(Signals2), I, LeaveOut

In [ ]:
#Initialize ICA methods and classifiers for experiments.

def ICA_METHODS(constrained=0, k=0, eps=1e-14):
    '''
    Parameters "constrained" and "k" regulates the maximum number of itterations allowed
    and used for the timeconstrained pipeline.
    Parameter "eps" controls the tolerance and is used for tolerance pipeline.
    
    Returns ICA methods as instances of classess. 
    '''
    return {
        "CoroICA":CoroICA(partitionsize= int(20 * 200),
                          max_iter =6000*(1-constrained) +2*k,
                          n_components=n_c,
                          groupsize = int(520*750),
                          minimize_loss=False,
                          condition_threshold=1000,
                          tol=eps,
                          pairing='neighbouring'),
        "choiICA": UwedgeICA(partitionsize= int(20 * 200),
                             max_iter =1000*(1-constrained) +3*k,
                             n_components=n_c,
                             condition_threshold=1000,
                             tol=eps,
                             instantcov=True,
                             timelags=None),
        "FastICA":FastICA(n_components=n_c,
                          tol=0.00001,
                          max_iter =(40*(1-constrained)+3*constrained+k)
                         )
    }


def Classifier():
    return {
        "SVM": svm.SVC(decision_function_shape='ovo', kernel = "linear"),
        "RF": RandomForestClassifier(n_estimators = 10000,max_depth = 5),
        "LDA":LinearDiscriminantAnalysis(),
        "QDA":QuadraticDiscriminantAnalysis(reg_param=0.1),
        "AdaBoost":AdaBoostClassifier(svm.SVC(decision_function_shape='ovo', kernel = "linear"),
                                      n_estimators=200),
        "MLP": MLPClassifier(random_state=1, max_iter=10000),
        "RFsLDA": RFsLDA()
    }

In [ ]:
# Run ICA on the observed mixed signals for the training data and recover estimated source signals.

def ICA_pipeline(method,Signals1,Signals2,I,LeaveOut,classifier = "SVM",
                constrained=0,k=0,eps=1e-14):
        ## Apply ICA
        transformer = ICA_METHODS(constrained,k,eps)[method]
        start = default_timer()  #Record start of timing
        X = transformer.fit_transform(Signals1)
        Y = transformer.transform(Signals2)
        end = default_timer()    #Record end of timing
        
        X,Y = X.transpose(), Y.transpose()
        X,Y = X.reshape((n_c, len(I), 750)),Y.reshape(n_c,len(N_list)-len(I),750)
        
        #Apply bandpass filters
        Filt1,Filt2 = lfilter(b,a,X),lfilter(b,a,Y)

        ## Extract bandpower features
        Features1,Features2 = np.log(np.var(Filt1, axis = 2)),np.log(np.var(Filt2, axis = 2))
        Features1,Features2 = Features1.transpose(),Features2.transpose()
        
        ## Fit Classifier and predict
        clf = Classifier()[classifier]
        clf.fit(Features1, Classes[I])

        predicts = clf.predict(Features2)
        return np.mean(predicts==Classes[LeaveOut]),end - start

In [ ]:
def Main_Pipeline(n_train=[8], max_iter = 5, echo = False,
                  classifier = "SVM", n_subs=9):
    df = pd.DataFrame(columns = ["Accuracy", "Method", "n_train", "Time"])    
    for n in n_train:
        cache = []
        n_subsets = min(comb(n_subs,n,exact=True),max_iter)
        for i in tqdm(np.random.permutation(list(combinations(range(n_subs),n)))[:n_subsets].tolist()):
            cache.append(i)

            S1,S2,I,L = Signals_pipeline(i)
            if echo:
                print("Finding ICA's for subset", i)
            try:
                x1, x2 = ICA_pipeline("FastICA",S1,S2,I,L,classifier) #Sometimes FastICA will crash.
            except ValueError:
                print("FastICA crashed")
                continue
            else:
                y1, y2 = ICA_pipeline("CoroICA",S1,S2,I,L,classifier)
                z1, z2 = ICA_pipeline("choiICA",S1,S2,I,L,classifier)

                df = df.append({"Accuracy":x1,"Method" : "FastICA","n_train":n,"Time":x2},ignore_index=True)
                df = df.append({"Accuracy":y1,"Method" : "CoroICA","n_train":n,"Time":y2},ignore_index=True)
                df = df.append({"Accuracy":z1,"Method":"choiICA","n_train":n,"Time":z2},ignore_index=True)
        print(cache)
            
    return df

## Try ICA on subject 1

In [ ]:
Sig1,Sig2, I, L = Signals_pipeline(range(1))

In [ ]:
transformer = FastICA(n_components=n_c, tol=0.01,random_state=0)
X1 = transformer.fit_transform(Sig1)
print(X1.shape)
X2 =  transformer.transform(Sig2)

In [ ]:
X1,X2 = X1.transpose(), X2.transpose()
X1,X2 = X1.reshape((n_c, len(I), 750)),X2.reshape(n_c,len(L),750)

### Bandpass filter and bandpower features

In [ ]:
#Apply bandpass filters
Filt1,Filt2 = lfilter(b,a,X1),lfilter(b,a,X2)

## Extract bandpower features
Features1,Features2 = np.log(np.var(Filt1, axis = 2)),np.log(np.var(Filt2, axis = 2))
Features1,Features2 = Features1.transpose(),Features2.transpose()

### Try classifying on ica's banpowers

In [ ]:
clf = Classifier()["SVM"]
clf.fit(Features1, Classes[I])

In [ ]:
predicts = clf.predict(Features2)
np.mean(predicts==Classes[L])

In [ ]:
clf = Classifier()["QDA"]
clf.fit(Features1, Classes[I])

In [ ]:
predicts = clf.predict(Features2)
np.mean(predicts==Classes[L])

# Main experiment

In [ ]:
#df = Main_Pipeline(range(1,9),max_iter=30, classifier = "SVM")  #FastICA will give warnings of convergence.

In [ ]:
df = pd.read_pickle("SVM30rep.pkl")

### Plots of main experiment

In [ ]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
fig, ax = plt.subplots(figsize=(15, 6), sharex=True)
ax = sns.violinplot(x="n_train", y="Accuracy", hue="Method", inner = "box",
                     palette="Set3", cut=1, bw = 1.5,
                     data=df,ax = ax)
ax.set(xlabel='Number of training subjects', ylabel='Prediction accuracy')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6), sharex=True)
ax = sns.lineplot(x="n_train", y="Accuracy", hue = "Method", estimator=np.median,
                  palette="Set3", err_kws = {"alpha": 0.3, "lw":2}, sizes = 3,
                  **{"fillstyle":"none"},
                  data=df) 
ax.set(xlabel='Number of training subjects', ylabel='Prediction accuracy')
plt.show()

In [ ]:
sns.set()
g = sns.FacetGrid(df, col="n_train", hue="Method",col_wrap = 4,
                  sharex=True, sharey=True)

ax = g.map(sns.scatterplot, "Time", "Accuracy")

# Time-constrained experiments

In [ ]:
def Time_Pipeline(n_train=6,k_max = 15, n_sets = 5,classifier = "SVM", n_subs=9):
    df = pd.DataFrame(columns = ["Accuracy", "Method", "Time", "k", "Subset"])    
    cache = []
    
    n_subsets = min(comb(n_subs,n_train,exact=True),n_sets)
    for i in tqdm(np.random.permutation(list(combinations(range(n_subs),n_train)))[:n_subsets].tolist()):
        cache.append(i)

        S1,S2,I,L = Signals_pipeline(i)
        for k in tqdm(range(1,k_max)):
            try:
                x1, x2 = ICA_pipeline("FastICA",S1,S2,I,L,classifier,1,k)
            except ValueError:
                print("FastICA crashed")
                continue
            else:
                y1, y2 = ICA_pipeline("CoroICA",S1,S2,I,L,classifier,1,k)
                z1, z2 = ICA_pipeline("choiICA",S1,S2,I,L,classifier,1,k)
                
                subs = "".join([str(c) for c in i])
                df = df.append({"Accuracy":x1,"Method":"FastICA","Time":x2,"k":k,"Subset":subs},ignore_index=True)
                df = df.append({"Accuracy":y1,"Method":"CoroICA","Time":y2,"k":k,"Subset":subs},ignore_index=True)
                df = df.append({"Accuracy":z1,"Method":"choiICA","Time":z2,"k":k,"Subset":subs},ignore_index=True)            
    return df

In [ ]:
df = Time_Pipeline(n_train=7,k_max = 30, n_sets=8, classifier = "SVM")

### Plots of time-constrained experiment

In [ ]:
ax = sns.lmplot(y="Accuracy", x="Time",hue="Method",sharex=True, sharey=True,
                col_wrap = 2, col = "Subset",
                lowess = True,
                data=df)

# Plot signals

In [ ]:
import matplotlib.backends.backend_pdf

def plot1(save=False):
    pdf = matplotlib.backends.backend_pdf.PdfPages("Signalsplot.pdf")
    
    
    fig1, axs = plt.subplots(5,1,figsize=(12, 6),sharex = True)
    #fig.tight_layout()
    for ax, sig in zip(axs,[Signals[0,i,:] for i in range(5)]):
        ax.plot(sig[:750])
    fig1.suptitle('Raw signals', fontsize=16)
    pdf.savefig(fig1)
    
    fig2, axs = plt.subplots(5,1,figsize=(12, 6),sharey =True,sharex=True)
    #fig.tight_layout()
    for ax, sig in zip(axs,[X1[0,i,:] for i in range(5)]):
        ax.plot(sig[:750])
    fig2.suptitle("Independent components", fontsize=16)
    pdf.savefig(fig2)
    
    fig3, axs = plt.subplots(5,1,figsize=(12, 6),sharey =True,sharex=True)
    #fig.tight_layout()
    for ax, sig in zip(axs,[Filt1[0,i,:] for i in range(5)]):
        ax.plot(sig[:750])
    fig3.suptitle("Filtered independent components", fontsize=16)
    if save:
        pdf.savefig(fig3)
    pdf.close()
    plt.show()

In [ ]:
plot1() #Requires having run the codesnippets under "Try ICA on subject 1"

# Test early stopping

In [ ]:
def Stopping_Pipeline(n_train=6,eps_max = 15, n_sets = 5,classifier = "SVM", n_subs=9):
    df = pd.DataFrame(columns = ["Accuracy", "Method", "Time", "tol", "Subset"])    
    cache = []
    
    n_subsets = min(comb(n_subs,n_train,exact=True),n_sets)
    for i in tqdm(np.random.permutation(list(combinations(range(n_subs),n_train)))[:n_subsets].tolist()):
        cache.append(i)

        S1,S2,I,L = Signals_pipeline(i)
        for l in tqdm(range(1,eps_max)):
            y1, y2 = ICA_pipeline("CoroICA",S1,S2,I,L,classifier,0,0,10**(-l))
            subs = "".join([str(c) for c in i])
            df = df.append({"Accuracy":y1,"Method":"CoroICA","Time":y2,
                            "tol":10**(-l),"Subset":subs},ignore_index=True)
            y1, y2 = ICA_pipeline("choiICA",S1,S2,I,L,classifier,0,0,10**(-l))
            df = df.append({"Accuracy":y1,"Method":"choiICA","Time":y2,
                            "tol":10**(-l),"Subset":subs},ignore_index=True)
    return df

In [ ]:
df = Stopping_Pipeline(n_train=4,eps_max = 14, n_sets=40, classifier = "SVM")

In [ ]:
df["-log(tol)"] = -np.log(df["tol"])

Plot of tolerance experiment

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6), sharex=True)
ax = sns.lineplot(x="-log(tol)", y="Accuracy", sort = False,
                  hue = "Method",
                  palette="Set2", err_kws = {"alpha": 0.3, "lw":2}, sizes = 3,
                  **{"fillstyle":"none"},
                  data=df) 
ax.set(xlabel='- log(Tolerance)', ylabel='Prediction accuracy')